<a href="https://colab.research.google.com/github/dlmacedo/deep-learning-class/blob/master/notebooks/IMDB_Classification_using_SimpleTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setups

https://wandb.ai/wandb/gallery/reports/SimpleTransformers-Transformers-Made-Easy--VmlldzoyNDQzNTg

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 4.6MB/s 
     |████████████████████████████████| 317kB 8.3MB/s 
     |████████████████████████████████| 890kB 10.8MB/s 
     |████████████████████████████████| 7.2MB 21.9MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 3.0MB 38.5MB/s 
     |████████████████████████████████| 1.6MB 59.9MB/s 
     |████████████████████████████████| 890kB 58.3MB/s 
     |████████████████████████████████| 1.1MB 56.3MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 112kB 59.7MB/s 
     |████████████████████████████████| 4.4MB 50.9MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 163kB 60.6MB/s 
     |████████████████████████████████| 122kB 65.1MB/s 
     |████████████████████████████████| 122kB 53.1MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb

wandb.login()

# setup wandb environment variables
%env WANDB_ENTITY=wandb
%env WANDB_PROJECT=gallery

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_ENTITY=wandb
env: WANDB_PROJECT=gallery


In [ ]:
from simpletransformers.classification import ClassificationModel
import logging

import sklearn
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# Download and Prepare Dataset

#### Download IMDB dataset

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2020-09-17 15:29:08--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  18.1MB/s    in 5.9s    

2020-09-17 15:29:14 (13.6 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



#### Helper function to read and prepare dataset

In [ ]:
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

#### Prepare train and validation split

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

#### Train and Evaluation data needs to be in a Pandas Dataframe.

In [ ]:
train_data = list(zip(train_texts, train_labels))
test_data = list(zip(test_texts, test_labels))

train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Fine-tune transformer model for binary text classification

Using SimpleTransformer it get's easier to experiment with different transformer. We just need three lines of code. 

#### Define train configs and define pretrained transformer model.

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_args={
    'num_train_epochs': 3,
    'train_batch_size': 16,
    'eval_batch_size': 64,
    'warmup_steps': 500,
    'weight_decay': 0.01,
    'logging_steps': 10,
    'learning_rate': 5e-5,
    'fp16': False,
    'wandb_project': 'gallery',
    "wandb_kwargs": {'entity': 'wandb'}
}

# Create a ClassificationModel
model = ClassificationModel('distilbert', 'distilbert-base-uncased', use_cuda=True, cuda_device=0, args=train_args) 

INFO:filelock:Lock 140262951256976 acquired on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


INFO:filelock:Lock 140262951256976 released on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


INFO:filelock:Lock 140260614338488 acquired on /root/.cache/torch/transformers/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock


INFO:filelock:Lock 140260614338488 released on /root/.cache/torch/transformers/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

INFO:filelock:Lock 140260606863960 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


#### Easy to fine-tune transformer using `model.train_model`.

In [ ]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:304: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: iit-bhu (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.1
wandb: Run data is saved locally in wandb/run-20200917_154546-23cnf4rb
wandb: Syncing run sweet-mountain-19


...

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:776: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


wandb: Waiting for W&B process to finish, PID 294
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200917_154546-23cnf4rb/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200917_154546-23cnf4rb/logs/debug-internal.log
wandb: Run summary:
wandb:                                                       Training loss 0.2767826020717621
wandb:                                                                  lr 0.0
wandb:                                                         global_step 3750
wandb:                                                               _step 374
wandb:                                                            _runtime 446
wandb:                                                          _timestamp 1600357993
wandb: Run history:
wandb:   Training loss ▆▆▃▅▄▄▄▇▆▆▂▃▃▃▃▅▂▁▄▄▃▄▃▂▂▃▃▂▃▁▁▁▃▂▁▁▁▁█▁
wandb:              lr ▂▃▄▆▇████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
wandb:     global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇

wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7274687308324573, 'tp': 10988, 'tn': 10601, 'fp': 1899, 'fn': 1512, 'acc': 0.86356, 'eval_loss': 0.5374512464532157}


#### Finally evaluate the model and get classification report.

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:776: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


wandb: Waiting for W&B process to finish, PID 327
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200917_134809-1r7st542/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200917_134809-1r7st542/logs/debug-internal.log
wandb: Run summary:
wandb:                                                       Training loss 0.005941650830209255
wandb:                                                                  lr 0.0
wandb:                                                         global_step 3750
wandb:                                                               _step 374
wandb:                                                            _runtime 486
wandb:                                                          _timestamp 1600350976
wandb: Run history:
wandb:   Training loss █▇▆▅▃▄▆▄▄▆▄▃▃▄▆▁▃▃▄▅▃▂▂▃▂▅▄▇▁▁▄▁▁▁▁▅▁▂▃▁
wandb:              lr ▂▃▄▆▇████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
wandb:     global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇

wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7363584876659166, 'tp': 11045, 'tn': 10655, 'fp': 1845, 'fn': 1455, 'acc': 0.868, 'eval_loss': 0.535767882462124}
